In [ ]:
import tensorflow as tf
from google.cloud.aiplatform.schema import dataset
from sympy import sequence

print('GPU:', tf.config.list_physical_devices('GPU'))

In [2]:
path_to_file = r"corpus.txt"

with open(path_to_file, 'r', encoding='utf-8') as f:
    text = f.read()

print(text)

我 喜欢 深度 学习
深度 学习 改变 世界
机器 学习 很 有 趣
人工 智能 应用 很 广泛
我们 明天 去 上班
程序 设计 很 有 趣
天气 很 好
天气 不 太 好
今天 心情 不错
明天 可能 会 下雨
我们 在 学习 深度 学习
模型 训练 需要 数据
数据 越 多 效果 越 好
我 喜欢 学习 新 知识
程序 员 每天 写 代码
代码 写 多 了 会 进步
学习 让 人 变 得 更 强大
深度 学习 模型 很 强大
自然 语言 处理 很 有 趣
我们 一起 来 学习
你 今天 心情 怎么样
机器 学习 和 深度 学习 有 关系
优化 模型 需要 耐心
人工 智能 技术 发展 很 快
程序 设计 是 一 门 艺术
今天 适合 学习
明天 继续 努力



In [3]:
# 文本长度
print(f'Length of text: {len(text)} characters')


Length of text: 336 characters


In [4]:
import re
# 句子末尾加入<eos>，让句子结束
my_corpus = re.sub(r'\n+', ' <eos> ', text).strip()
print(my_corpus)

我 喜欢 深度 学习 <eos> 深度 学习 改变 世界 <eos> 机器 学习 很 有 趣 <eos> 人工 智能 应用 很 广泛 <eos> 我们 明天 去 上班 <eos> 程序 设计 很 有 趣 <eos> 天气 很 好 <eos> 天气 不 太 好 <eos> 今天 心情 不错 <eos> 明天 可能 会 下雨 <eos> 我们 在 学习 深度 学习 <eos> 模型 训练 需要 数据 <eos> 数据 越 多 效果 越 好 <eos> 我 喜欢 学习 新 知识 <eos> 程序 员 每天 写 代码 <eos> 代码 写 多 了 会 进步 <eos> 学习 让 人 变 得 更 强大 <eos> 深度 学习 模型 很 强大 <eos> 自然 语言 处理 很 有 趣 <eos> 我们 一起 来 学习 <eos> 你 今天 心情 怎么样 <eos> 机器 学习 和 深度 学习 有 关系 <eos> 优化 模型 需要 耐心 <eos> 人工 智能 技术 发展 很 快 <eos> 程序 设计 是 一 门 艺术 <eos> 今天 适合 学习 <eos> 明天 继续 努力 <eos>


In [5]:
# 分词+ 编码
from tensorflow.keras.preprocessing.text import Tokenizer

#原始文本变成“整数编号”序列
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts([my_corpus])
tokens = tokenizer.texts_to_sequences([my_corpus])[0]
print(tokens)

seq_len = 15
batch_size = 128
vocab_size= 1000


[13, 14, 4, 2, 1, 4, 2, 29, 30, 1, 15, 2, 3, 5, 6, 1, 16, 17, 31, 3, 32, 1, 7, 8, 33, 34, 1, 9, 18, 3, 5, 6, 1, 19, 3, 10, 1, 19, 35, 36, 10, 1, 11, 20, 37, 1, 8, 38, 21, 39, 1, 7, 40, 2, 4, 2, 1, 12, 41, 22, 23, 1, 23, 24, 25, 42, 24, 10, 1, 13, 14, 2, 43, 44, 1, 9, 45, 46, 26, 27, 1, 27, 26, 25, 47, 21, 48, 1, 2, 49, 50, 51, 52, 53, 28, 1, 4, 2, 12, 3, 28, 1, 54, 55, 56, 3, 5, 6, 1, 7, 57, 58, 2, 1, 59, 11, 20, 60, 1, 15, 2, 61, 4, 2, 5, 62, 1, 63, 12, 22, 64, 1, 16, 17, 65, 66, 3, 67, 1, 9, 18, 68, 69, 70, 71, 1, 11, 72, 2, 1, 8, 73, 74, 1]


In [6]:
#80 训练集 ，20% 测试集
split = int(len(tokens) * 0.8)
if len(tokens) - split < seq_len + 1:
    split = len(tokens) - seq_len - 1
train_tok = tokens[:split]
test_tok  = tokens[split:]

In [7]:
# 把文本向量转换为字符索
def make_dataset(tokens, seq_len, batch_size):
    windows, labels = [], []
    for i in range(seq_len, len(tokens)):
        windows.append(tokens[i-seq_len:i])
        labels.append(tokens[i])
    ds = tf.data.Dataset.from_tensor_slices((windows, labels))
    return ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = make_dataset(train_tok, seq_len, batch_size)
test_ds = make_dataset(test_tok, seq_len, batch_size)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


model = Sequential([
     Embedding(vocab_size, 128),
     LSTM(128),
     Dense(vocab_size, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         128000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1000)              129000    
                                                                 
Total params: 388,584
Trainable params: 388,584
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 训练
model.fit(train_ds,
          epochs=200,
          callbacks=[EarlyStopping(patience=5, restore_best_weights=True),
                     ReduceLROnPlateau(patience=3, factor=0.5)]
          )

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 6.9075 - accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/200
1/1 [==============================] - 0s 11ms/step - loss: 6.9015 - accuracy: 0.1019 - lr: 0.0010
Epoch 3/200
1/1 [==============================] - 0s 9ms/step - loss: 6.8952 - accuracy: 0.1852 - lr: 0.0010
Epoch 4/200
1/1 [==============================] - 0s 10ms/step - loss: 6.8879 - accuracy: 0.1759 - lr: 0.0010
Epoch 5/200
1/1 [==============================] - 0s 11ms/step - loss: 6.8790 - accuracy: 0.1759 - lr: 0.0010
Epoch 6/200
1/1 [==============================] - 0s 9ms/step - loss: 6.8674 - accuracy: 0.1759 - lr: 0.0010
Epoch 7/200
1/1 [==============================] - 0s 10ms/step - loss: 6.8518 - accuracy: 0.1759 - lr: 0.0010
Epoch 8/200
1/1 [==============================] - 0s 9ms/step - loss: 6.8301 - accuracy: 0.1759 - lr: 0.0010
Epoch 9/200
1/1 [==============================] - 0s 9ms/step - loss: 6.7992 - accuracy: 0.1759 - lr: 0.0010
Epo

In [10]:
#评估
test_loss, test_acc = model.evaluate(test_ds, verbose=1)   # ← 顺序修正
print(f"loss = {test_loss:.4f}")
print(f"acc  = {test_acc:.4f}")

1/1 [==============================] - 0s 284ms/step - loss: 7.3144 - accuracy: 0.2500
loss = 7.3144
acc  = 0.2500


In [ ]:
#保存
model.save('model.h5')

In [13]:
# 单步预测下一个词
import numpy as np
def predict_next(model, tokenizer, prefix, temperature=1.0, max_retry=5):
    for _ in range(max_retry):
        seq = tokenizer.texts_to_sequences([prefix.split()])[0]
        seq = tf.keras.preprocessing.sequence.pad_sequences([seq], maxlen=seq_len, padding='pre')
        logits = model(seq, training=False)[0]
        if temperature != 1.0:
            logits /= temperature
        prob = tf.nn.softmax(logits).numpy()
        next_id = np.random.choice(len(prob), p=prob)
        next_word = tokenizer.index_word.get(next_id, '<unk>')
        if next_word != '<unk>':
            return next_word
    # 重试失败 → 回退到最高概率词
    return tokenizer.index_word.get(np.argmax(prob), '<unk>')






In [12]:
#  循环生成多词
def generate(model, tokenizer, prefix, max_tokens=20, temperature=0.8):
    for _ in range(max_tokens):
        nxt = predict_next(model, tokenizer, prefix, temperature)
        if nxt == '<eos>':          # 遇到结束符就停
            break
        prefix += ' ' + nxt
    return prefix

# 生成 15 个词
print("生成结果：", generate(model, tokenizer, "深度 学习", max_tokens=15, temperature=0.8))

生成结果： 深度 学习 <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 我 <unk> <unk> <unk>
